In [2]:
from my_functions import*

In [3]:
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root") #opening the Root file with Uproot 
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"]
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]

In [4]:
#Prepare the data for the plots

#Choose the range of events to plot
nmin=0
nmax=100

#Select quality 0 Z->mumu
Zmumu_pt=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_pt"].array(),0)[nmin:nmax]
Zmumu_eta=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_eta"].array(),0)[nmin:nmax]
Zmumu_phi=quality_selector(MuonTree_Zmumu["muon_quality"].array(),MuonTree_Zmumu["muon_phi"].array(),0)[nmin:nmax]

#And select the Z peak pairs
Zmumu_pt, Zmumu_eta, Zmumu_phi = get_all_Z_peak_pairs(Zmumu_pt,Zmumu_eta,Zmumu_phi)

#Select the ZeroBias data with energy cut
ZeroBias_eta=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_eta"].array())[nmin:nmax]
ZeroBias_phi=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_phi"].array())[nmin:nmax]
ZeroBias_pt=energy_cut(MuonTree_ZeroBias["muon_pt"].array(), MuonTree_ZeroBias["muon_pt"].array())[nmin:nmax]

quality_selector: Only 21.265 % of the data has been selected
quality_selector: Only 21.265 % of the data has been selected
quality_selector: Only 21.265 % of the data has been selected


100%|██████████| 100/100 [00:00<00:00, 1534.36it/s]

energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut
energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut
energy_cut: Only 5.79% of the events have survived the cut with a lower cut of 14000 and no upper cut


In [5]:
Zmumu_range=[0, nmax]
ZeroBias_range=[0, nmax]
bins=np.linspace(0,1,250)


In [ ]:
dr_list=[]
FPR_list=[]

i=50
j=20
for val in range(i):
    mins, maxs = generate_random_float_pairs(0.0, 0.8, j, min_diff=0.2)
    dr_min=mins
    dr_max=maxs

    dr, FPR_value =ROC_curve_efficiency_optimiser(MuonTree_Zmumu, MuonTree_ZeroBias, Zmumu_pt, Zmumu_eta, Zmumu_phi, ZeroBias_pt, ZeroBias_eta, ZeroBias_phi,
                                Zmumu_range, ZeroBias_range, bins, dr_min, dr_max)
    dr_list.append(dr)
    FPR_list.append(FPR_value)

k=i*j
min_FPR=min(FPR_list)
best_dr=dr_list[FPR_list.index(min_FPR)]
print(f"After", i*j ,f"iterations, the best delta R range is: {best_dr} with an FPR of {min_FPR}")

Computing ROC curve:   0%|          | 0/1 [00:00<?, ?it/s].29s/it]